# Important! You must generate a Specklia access key before running this workbook

This example accesses Earthwave's Specklia point data service. This does not require user registration but it does require that each user generates an API access token via:
https://specklia.earthwave.co.uk/

More information on the Specklia platform can be found in the documentation here: https://specklia.earthwave.co.uk/static/docs/index.html

Once an API key has been generated, please save it in a file named "specklia_api_key.txt".

### Prepare the environment

Installing some additional packages not installed in the conda environment by default.

In [ ]:
!pip3 install --upgrade specklia
!pip3 install matplotlib
!pip3 install contextily

### Import the client library and create an instance of the client with a key obtained from https://specklia.earthwave.co.uk/

In [ ]:
import sys
sys.path.append("./.local/lib/python3.10/site-packages")
from specklia import Specklia

with open("specklia_api_key.txt", "r") as file:
    api_key = file.read()

# Temporary non-official url for demo!
client = Specklia(api_key)

### Example: query the client for available datasets

In [ ]:
datasets = client.list_datasets()
datasets

### Now query the client for the CryoSat-2 Gridded EOLIS Elevation Product

In [ ]:
import shapely
from datetime import datetime

svalbard_polygon = shapely.Polygon(([15, 81], [28, 80.5], [29, 79.03], [14.57, 78.9], [15, 81]))


dataset_name = 'CryoTEMPO-EOLIS Gridded Product'
available_datasets = client.list_datasets()
gridded_product_dataset = available_datasets[
    available_datasets['dataset_name'] == dataset_name].iloc[0]

gridded_product_data_2012, sources = client.query_dataset(
    dataset_id=gridded_product_dataset['dataset_id'],
    epsg4326_polygon=svalbard_polygon,
    min_datetime=datetime(2012, 1, 1),
    max_datetime=datetime(2012, 1, 31, 23, 59, 59),
    columns_to_return=['timestamp', 'elevation', 'x', 'y'],
    additional_filters=[],
    source_information_only=False)

print(
    f'Query complete, {len(gridded_product_data_2012)} points returned, drawn from {len(sources)} original sources.')

### Convert the dataframe to a 2-D Array

In [ ]:
import pandas as pd
import numpy as np

source_information = sources[0]['source_information']
resolution = source_information['geospatial_resolution']
projection = source_information['geospatial_projection']

df = gridded_product_data_2012.copy(deep=True)
df = df[['x','y','elevation']]
df['elevation'] = df['elevation'].astype(float)

point_extent = (df.x.min(),
                df.y.min(),
                df.x.max(),
                df.y.max())
array_extent = (point_extent[0] - resolution / 2,
                point_extent[1] - resolution / 2,
                point_extent[2] + resolution / 2,
                point_extent[3] + resolution / 2)

# Build unique x and y points
xs = np.arange(point_extent[0], point_extent[2] + 1, resolution)
ys = np.arange(point_extent[3], point_extent[1] - 1, -resolution)

xs, ys = np.repeat(xs, len(ys)), np.tile(ys, len(xs))
df_grid = pd.DataFrame(data={'x': xs, 'y': ys})

df_dense = pd.merge(df_grid, df, on=['x', 'y'], how='left')

data_array = df_dense.pivot(index='y', columns='x', values='elevation').values
data_array = data_array[::-1]

### View the data with a simple plot

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import contextily as ctx

fig, ax = plt.subplots()
im = ax.imshow(data_array, cmap='viridis', extent=[array_extent[0], array_extent[2], array_extent[1], array_extent[3]], interpolation='none', origin='upper')
# ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, crs=3413)

ax.set_xlabel("Easting (m)")
ax.set_ylabel("Northing (m)")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
cb = fig.colorbar(im, ax=ax, cax=cax)
cb.set_label('Elevation (m)')
fig.suptitle('CryoSat-2 EOLIS Gridded Elevation\nProduct for Austfonna Svalbard', fontweight='bold')


### Save in an xcube compatible format

In [ ]:
import netCDF4
from rasterio.crs import CRS
import numpy as np
import xarray as xr
from xcube.core.chunk import chunk_dataset
from xcube.core.store import new_data_store

# EPSG:3413
proj4_code = "+proj=stere +lat_0=90 +lat_ts=70 +lon_0=-45 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs"

# resolution = resolution
bounding_box = array_extent

data = data_array

with netCDF4.Dataset('tmp.nc', 'w', diskless=True) as nc:

    # Create dimensions
    nc.createDimension('y', data.shape[0])
    nc.createDimension('x', data.shape[1])

    # Create variables
    y_var = nc.createVariable('y', np.float64, ('y'))
    # lat_var.units = 'metres_north'
    y_var.standard_name = 'North'
    y_var.axis = 'y'

    x_var = nc.createVariable('x', np.float64, ('x'))
    # lon_var.units = 'metres_east'
    x_var.standard_name = 'East'
    x_var.axis = 'x'

    var = nc.createVariable('data', "float32", ('y', 'x'), fill_value=9999)
    var.units = 'm'
    var.long_name = 'EOLIS Elevation'
    var.short_name = 'EOLIS Elevation'

    # Set up CRS
    if proj4_code is not None:
        crs_gris = CRS.from_proj4(proj4_code)
        nc_crs = nc.createVariable('spatial_ref', 'i4')
        nc_crs.spatial_ref = crs_gris.to_wkt()
        var.grid_mapping = 'spatial_ref'

    y = np.arange(bounding_box[1] + resolution / 2.0, bounding_box[3], resolution)
    x = np.arange(bounding_box[0] + resolution / 2.0, bounding_box[2], resolution)

    # Flip axis if y-affine negative.
    # y_var[:] = y[::-1]
    y_var[:] = y

    x_var[:] = x
    var[:, :] = data

    chunk_size=256
    store = new_data_store('file', root='./')
    dataset = xr.open_dataset(xr.backends.NetCDF4DataStore(nc))

    dataset.variables['data'].attrs['4d_viewer_layer_type'] = 'heatmap'  # or heightmap
    dataset.attrs['4d_viewer_ui_path'] = '/PolarTEP/'

    chunked_dataset = chunk_dataset(dataset, dict(y=chunk_size, x=chunk_size), format_name='zarr')

    base_name = "elevation-cryosat-2"
    store.write_data(chunked_dataset, f'{base_name}.zarr', replace=True)
    store.write_data(chunked_dataset, f"{base_name}.levels", replace=True, base_dataset_id=f'{base_name}.zarr', agg_methods="first")